In [1]:
import pandas as pd
import re
import itertools
import networkx as nx

In [2]:
df = pd.read_csv('./data/film_festivals.csv', keep_default_na=False)

In [3]:
df = df[~(df['Director'].isin(['N/A', '']) & df['Writer'].isin(['N/A', '']) & df['Actors'].isin(['N/A', '']))]
df['Director'] = df['Director'].apply(lambda row: re.sub('\(.*?\)', '', row))
df['Writer'] = df['Writer'].apply(lambda row: re.sub(' \(.*?\)', '', row))
df = df.reset_index(drop=True)
df.head()

id      title_english title_original         director        country  \
0   0    Adieu Bonaparte  وداعا بونابرت  Youssef Chahine          Egypt   
1   1              Birdy            N/A      Alan Parker  United States   
2   2              Bliss            N/A     Ray Lawrence      Australia   
3   4  The Coca-Cola Kid            N/A  Dušan Makavejev      Australia   
4   5       Colonel Redl    Oberst Redl     István Szabó        Hungary   

   winner festival  year_festival  \
0       0   cannes           1985   
1       0   cannes           1985   
2       0   cannes           1985   
3       0   cannes           1985   
4       0   cannes           1985   

                                         link_film  \
0    https://en.wikipedia.org/wiki/Adieu_Bonaparte   
1       https://en.wikipedia.org/wiki/Birdy_(film)   
2  https://en.wikipedia.org/wiki/Bliss_(1985_film)   
3  https://en.wikipedia.org/wiki/The_Coca-Cola_Kid   
4       https://en.wikipedia.org/wiki/Colonel_Redl   

                                       link_director  ... BoxOffice  \
0      https://en.wikipedia.org/wiki/Youssef_Chahine  ...       N/A   
1          https://en.wikipedia.org/wiki/Alan_Parker  ...       N/A   
2  https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...  ...       N/A   
3  https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...  ...       N/A   
4  https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...  ...       N/A   

                         Production Website Response  \
0                               N/A     N/A     True   
1  Sony Pictures Home Entertainment     N/A     True   
2           Starmaker Entertainment     N/A     True   
3                  Cinecom Pictures     N/A     True   
4                               N/A     N/A     True   

  InternetMovieDatabaseRating RottenTomatoesRating MetacriticRating Error  \
0                      6.4/10                  N/A              N/A         
1                      7.3/10                  85%              N/A         
2                      6.9/10                  N/A              N/A         
3                      6.0/10                  44%              N/A         
4                      7.6/10                  N/A              N/A         

  totalSeasons Ratings  
0                       
1                       
2                       
3                       
4                       

[5 rows x 40 columns]

In [4]:
df_cannes_2010_2019 = df[(df['festival'] == 'cannes') & (df['year_festival'] >= 2010)].reset_index(drop=True)
df_cannes_2010_2019[['Director', 'Writer', 'Actors']].head()

Director                                             Writer  \
0             Mike Leigh                                         Mike Leigh   
1  Alejandro G. Iñárritu  Alejandro G. Iñárritu, Alejandro G. Iñárritu, ...   
2       Nikita Mikhalkov  Nikita Mikhalkov, Vladimir Moiseenko, Aleksand...   
3       Abbas Kiarostami               Abbas Kiarostami, Caroline Eliacheff   
4         Xiaoshuai Wang                                         Yishu Yang   

                                              Actors  
0  Jim Broadbent, Ruth Sheen, Lesley Manville, Ol...  
1  Javier Bardem, Maricel Álvarez, Hanaa Bouchaib...  
2  Nikita Mikhalkov, Oleg Menshikov, Nadezhda Mik...  
3  Juliette Binoche, William Shimell, Jean-Claude...  
4        Bingbing Fan, Feier Li, Hao Qin, Xueqi Wang

In [5]:
def get_nodelist(directors_from_df, writers_from_df, actors_from_df):
    # directors
    directors = directors_from_df.split(', ')
    directors = [director for director in directors if director != 'N/A']
    directors = [(director, 'director') for director in directors]
    # writers
    writers = writers_from_df.split(', ')
    writers = [writer for writer in writers if writer != 'N/A']
    writers = [(writer, 'writer') for writer in writers]
    # actors
    actors = actors_from_df.split(', ')
    actors = [actor for actor in actors if actor != 'N/A']
    actors = [(actor, 'actor') for actor in actors]
    # people: directors, actors, and writers
    people = directors
    people.extend(actors) # the order of the extends determines the order of priority: director -> actor -> writer for people who have multiple roles
    people.extend(writers)
    
    name = [person[0] for person in people]
    role = [person[1] for person in people]
    nodelist = pd.DataFrame({'Name': name,
                             'Role': role})
    nodelist = nodelist.drop_duplicates(subset='Name').reset_index(drop=True)
    return nodelist

In [6]:
def get_edgelist(nodelist):
    edges = [combo + (1,) for combo in itertools.combinations(nodelist['Name'], 2)]
    edgelist = pd.DataFrame(edges, columns=['Source', 'Target', 'Weight'])
    return edges, edgelist

In [7]:
def get_film_edges(directors_from_df, writers_from_df, actors_from_df):
    nodelist_film = get_nodelist(directors_from_df,
                                 writers_from_df,
                                 actors_from_df)
    edges_film, edgelist_film = get_edgelist(nodelist_film)
    return edges_film    

In [8]:
def get_graph(df):
    df_directors = df['Director'].str.cat(sep=', ')
    df_writers = df['Writer'].str.cat(sep=', ')
    df_actors = df['Actors'].str.cat(sep=', ')
    
    df_nodelist = get_nodelist(df_directors,
                               df_writers,
                               df_actors)
    df_node_names = list(df_nodelist['Name'])
    
    df_edges = df.apply(lambda row: get_film_edges(row['Director'], row['Writer'], row['Actors']), axis=1)
    df_edges = df_edges.tolist()
    df_edges = [item for sublist in df_edges for item in sublist]
    
    M = nx.MultiGraph()
    M.add_nodes_from(df_node_names)
    M.add_weighted_edges_from(df_edges)
    
    # https://stackoverflow.com/questions/15590812/networkx-convert-multigraph-into-simple-graph-with-weighted-edges
    # create weighted graph from M
    G = nx.Graph() 
    for u,v,data in M.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if G.has_edge(u,v):
            G[u][v]['weight'] += w
        else:
            G.add_edge(u, v, weight=w)
    
    return G

In [9]:
G = get_graph(df_cannes_2010_2019)
# print(G.nodes(data=True))
print(G.edges(data=True))

[('Mike Leigh', 'Jim Broadbent', {'weight': 1}), ('Mike Leigh', 'Ruth Sheen', {'weight': 1}), ('Mike Leigh', 'Lesley Manville', {'weight': 1}), ('Mike Leigh', 'Oliver Maltman', {'weight': 1}), ('Mike Leigh', 'Timothy Spall', {'weight': 1}), ('Mike Leigh', 'Paul Jesson', {'weight': 1}), ('Mike Leigh', 'Dorothy Atkinson', {'weight': 1}), ('Mike Leigh', 'Marion Bailey', {'weight': 1}), ('Jim Broadbent', 'Ruth Sheen', {'weight': 1}), ('Jim Broadbent', 'Lesley Manville', {'weight': 1}), ('Jim Broadbent', 'Oliver Maltman', {'weight': 1}), ('Ruth Sheen', 'Lesley Manville', {'weight': 1}), ('Ruth Sheen', 'Oliver Maltman', {'weight': 1}), ('Lesley Manville', 'Oliver Maltman', {'weight': 1}), ('Timothy Spall', 'Paul Jesson', {'weight': 1}), ('Timothy Spall', 'Dorothy Atkinson', {'weight': 1}), ('Timothy Spall', 'Marion Bailey', {'weight': 1}), ('Paul Jesson', 'Dorothy Atkinson', {'weight': 1}), ('Paul Jesson', 'Marion Bailey', {'weight': 1}), ('Dorothy Atkinson', 'Marion Bailey', {'weight': 1}),

In [10]:
nx.write_gexf(G, './data/network/cannes_2010_2019.gexf')